In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
import numpy as np
import os

# Paths
download_path = "traffic_sign_dataset"
train_dir = os.path.join(download_path, "traffic_Data/DATA")
test_dir = os.path.join(download_path, "traffic_Data/TEST")

# Data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training",
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
)

# Pretrained ResNet50
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

# Model compilation
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the baseline model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
)

# Save the model
model.save("basemodel.h5")

Found 3359 images belonging to 58 classes.
Found 811 images belonging to 58 classes.
Epoch 1/10
 37/104 ━━━━━━━━━━━━━━━━━━━━ 7:33 7s/step - accuracy: 0.5275 - loss: 2.1859

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Dropout

# Adding Batch Normalization and Dropout layers to the baseline model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

# Model compilation
enhanced_model = Model(inputs=base_model.input, outputs=predictions)
enhanced_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the enhanced model
enhanced_history = enhanced_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
)

# Save the enhanced model
enhanced_model.save("enhancedmodel.h5")